<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Request-data-from-de-API" data-toc-modified-id="Request-data-from-de-API-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Request data from de API</a></span><ul class="toc-item"><li><span><a href="#Request-data-from-the-server-to-obtain-taxonomic-infomation:" data-toc-modified-id="Request-data-from-the-server-to-obtain-taxonomic-infomation:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Request data from the server to obtain taxonomic infomation:</a></span></li><li><span><a href="#Get-the-total-number-of-occurrences-for-each-species" data-toc-modified-id="Get-the-total-number-of-occurrences-for-each-species-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get the total number of occurrences for each species</a></span></li></ul></li><li><span><a href="#Saving-the-dataframe" data-toc-modified-id="Saving-the-dataframe-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Saving the dataframe</a></span></li></ul></div>

### Libraries 

pygbif is split up into modules for each of the major groups of API methods.

Registry - Datasets, Nodes, Installations, Networks, Organizations
Species - Taxonomic names
Occurrences - Occurrence data, including the download API
Maps - Maps, get raster maps from GBIF as png or mvt

In [1]:
import sys
sys.path.append('../')

In [2]:
from pygbif import occurrences as occ
from pygbif import species
import requests
import pandas as pd
import src.functions_API as fa

### Request data from de API

#### Request data from the server to obtain taxonomic infomation:

In [3]:
url = 'http://api.gbif.org/v1/species/match?verbose=true&name='
data = requests.get(url)
data.status_code

200

In order to call the species with the GBIF API, it is necessary to enter the codes for each species. As it is impossible to know the codes, I am going to generate a function that, giving the scientific name, will return the codes for each one of them.

In [4]:
sp = ['Apus affinis', 'Apus caffer', 'Silene ciliata','Abies pinsapo', 'Ciconia ciconia',
      'Anas penelope', 'Grus grus', 'Vanellus vanellus', 'Cortaderia selloana','Carpobrotus edulis', 'Alytes dickhilleni',
      'Parnassius apollo', 'Melitaea cinxia', 'Sonchus tenerrimus']

In [5]:
code_number = fa.get_species_name_from_codes(sp)
code_number

{'Apus affinis': 5228662,
 'Apus caffer': 5228644,
 'Silene ciliata': 7267056,
 'Abies pinsapo': 2685464,
 'Ciconia ciconia': 2481912,
 'Anas penelope': 2498111,
 'Grus grus': 2474950,
 'Vanellus vanellus': 2480242,
 'Cortaderia selloana': 2704523,
 'Carpobrotus edulis': 3084842,
 'Alytes dickhilleni': 2426613,
 'Parnassius apollo': 1938810,
 'Melitaea cinxia': 8168644,
 'Sonchus tenerrimus': 3105768}

#### Get the total number of occurrences for each species
We can use the function occ.search which search for GBIF occurences.

In [6]:
response = requests.get('http://api.gbif.org/v1/occurrence/counts/datasets')
response.status_code

200

In [7]:
sp_list = list(code_number.values())

In [8]:
%time
all_occurrences = []
for i in sp_list:
    y = fa.join_occurrences_taxonomic(i)
    all_occurrences.append(y)
    


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs


In [11]:
df = pd.concat(all_occurrences).fillna(0).reset_index(drop=True)
df

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
1,1971,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
2,1972,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
3,1973,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
4,1974,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
...,...,...,...,...,...,...,...,...,...
695,2015,143,Sonchus tenerrimus L.,Plantae,Sonchus,Asteraceae,Spain,HUMAN_OBSERVATION,US
696,2016,137,Sonchus tenerrimus L.,Plantae,Sonchus,Asteraceae,Spain,HUMAN_OBSERVATION,US
697,2017,50,Sonchus tenerrimus L.,Plantae,Sonchus,Asteraceae,Spain,HUMAN_OBSERVATION,US
698,2018,95,Sonchus tenerrimus L.,Plantae,Sonchus,Asteraceae,Spain,HUMAN_OBSERVATION,US


### Saving the dataframe 

In [12]:
df.to_csv("species.csv")